In [65]:
import os
import json
import rasterio
import urllib.request
import pystac
from datetime import datetime, timezone
from shapely.geometry import Polygon, mapping, shape
from tempfile import TemporaryDirectory
import logging
import constants
import boto3
from botocore.exceptions import ClientError
from glob import glob

In [66]:
#set up logging 
logging.basicConfig(level=logging.DEBUG)
debug=logging.debug
info=logging.info
warning=logging.warning
error=logging.error

In [67]:
# location for STAC jsons 
stac_loc=r"STAC_DEM/Catalog/"

#list DEM objects in bucket
object_key=r"STAC_DEM/Data/"


# # use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)
# for some reason the bucket is adding an extra letter at the end???
bucket = constants.AWS_S3_BUCKET



DEBUG:botocore.hooks:Event choose-service-name: calling handler <function handle_service_name_alias at 0x7f7b5b1c7b00>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x7f7b5b1513a0>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function lazy_call.<locals>._handler at 0x7f7b5b0cd440>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x7f7b5b151120>
DEBUG:botocore.endpoint:Setting s3 timeout as (60, 60)
DEBUG:botocore.client:Registering retry handlers for service: s3
DEBUG:botocore.utils:Registering S3 region redirector handler
DEBUG:botocore.utils:Registering S3Express Identity Resolver


In [70]:
def create_url(bucket_name: str,
            object_name: str):
 
    """
           
    This function takes a bucket name, an object name, and an expiration time (in seconds) and generates a URL download link for the object.

    Arguments:
        bucket_name: String of name of the bucket
        object_name: Name of the object (key) that the URL will be pointed to

    Returns:
        Link of output (object download) URL
        
    Raises: 
        Exceptions raised will display an error message and be logged in the export.log file
    """
    try:
        if r':443' in constants.AWS_S3_ENDPOINT:
            endpoint=constants.AWS_S3_ENDPOINT.split(':')
            endpoint=fr"{endpoint[0]}:{endpoint[1]}"
        else:
            endpoint=constants.AWS_S3_ENDPOINT
        response=os.path.join(endpoint,bucket_name,object_name)
    except ClientError as e:
        print(e)
        return None
    return response

def set_permissions(bucket_name: str,
                    object_name: str,
                    permissions='public-read'):
    """
    This function takes a bucket name, an object name, and a permissions value (specified below) and sets the object's permissions to the value given.

    Arguments:
        bucket_name: String of name of the bucket
        object_name: Name of the object (key) that the URL will be pointed to
        permissions: If not specified, the permissions will default to 'public-read'. Otherwise, permissions can be found below:
        'private'|'public-read'|'public-read-write'|'authenticated-read'|'aws-exec-read'|'bucket-owner-read'|'bucket-owner-full-control'

    Returns:
        Nothing
        
    Raises: 
        Exceptions raised will display an error message and be logged in the export.log file 
    """

    try:
        response = s3_client.put_object_acl(ACL=permissions, Bucket=bucket_name, Key=object_name)
        print(f'Set permissions on {object_name} success, set to {permissions}')
    except Exception as e:
        print(f'Error when setting permission: double check permission: {permissions}. Refer to help(set_permissions) for documentation.')
        print(e)

    return

#function to get data for json
def get_bbox_and_footprint(raster):
    with rasterio.open(raster) as r:
        bounds = r.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])
        
        return (bbox, mapping(footprint))

In [71]:
# find all DEM objects in s3 and add to a list, and check to make sure they have an object size 

DEM_Objects=[]

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=object_key, StartAfter=object_key)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        DEM_Objects.append(obj['Key'])
        info(obj['Key'])
        info(f"Object Size {obj['Size']}")
        
else:
    error("No objects found in the bucket.")


DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function set_list_objects_encoding_type_url at 0x7f7b5b1f3c40>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function validate_bucket_name at 0x7f7b5b1f27a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function remove_bucket_from_url_paths_from_model at 0x7f7b5b01c9a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <bound method S3RegionRedirectorv2.annotate_request_context of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <bound method ClientCreator._inject_s3_input_parameters of <botocore.client.ClientCreator object at 0x7f7b58b0b290>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function generate_idempotent_uuid at 0x7f7b5b1f25c0>
DEBUG:botocore.

In [72]:
#create base url for all json hrefs 
base_url=create_url(bucket, stac_loc)
catalog_name='DEM_Test'


In [73]:
#get all asset hrefs and set permissions to public 
dem_url=[]
for d in DEM_Objects:
    info(d)
    set_permissions(bucket, d) # default is public-read
    url=create_url(bucket, d)
    dem_url.append(url)

INFO:root:STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_2_cog.tif
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function validate_bucket_name at 0x7f7b5b1f27a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function remove_bucket_from_url_paths_from_model at 0x7f7b5b01c9a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <bound method S3RegionRedirectorv2.annotate_request_context of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <bound method ClientCreator._inject_s3_input_parameters of <botocore.client.ClientCreator object at 0x7f7b58b0b290>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function generate_idempotent_uuid at 0x7f7b5b1f25c0>
DEBUG:botocore.hooks:Event before-endpoint-resolution.s3: calling handler <function customize_endpoint_res

Set permissions on STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_2_cog.tif success, set to public-read
Set permissions on STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_cog.tif success, set to public-read
Set permissions on STAC_DEM/Data/bc_092o019_xli1m_utm10_2018_2_cog.tif success, set to public-read


DEBUG:urllib3.connectionpool:https://nrs.objectstore.gov.bc.ca:443 "PUT /cloudgistest/STAC_DEM/Data/bc_092o019_xli1m_utm10_2018_cog.tif?acl HTTP/1.1" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 04 Jul 2024 15:42:48 GMT', 'Server': 'ViPR/1.0', 'x-amz-request-id': '8e22ee15:19021c2fd34:45e4f:6353', 'x-amz-id-2': 'e18c9cbfeaa4fedc34641e6077f97d3c6046bd438c802f7b82cf0a456b3dc938', 'Content-Length': '0'}
DEBUG:botocore.parsers:Response body:
b''
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <botocore.retryhandler.RetryHandler object at 0x7f7b58b0eed0>
DEBUG:botocore.retryhandler:No retry needed.
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <bound method S3RegionRedirectorv2.redirect_from_error of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
INFO:root:STAC_DEM/Data/bc_092o020_xli1m_utm10_2018_2_cog.tif
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function validate_

Set permissions on STAC_DEM/Data/bc_092o019_xli1m_utm10_2018_cog.tif success, set to public-read
Set permissions on STAC_DEM/Data/bc_092o020_xli1m_utm10_2018_2_cog.tif success, set to public-read


DEBUG:urllib3.connectionpool:https://nrs.objectstore.gov.bc.ca:443 "PUT /cloudgistest/STAC_DEM/Data/bc_092o020_xli1m_utm10_2018_cog.tif?acl HTTP/1.1" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 04 Jul 2024 15:42:48 GMT', 'Server': 'ViPR/1.0', 'x-amz-request-id': '8e22ee15:19021c2fd34:45e4f:6358', 'x-amz-id-2': 'c40632a0a2b9c45b6212676c4876e43df047d154d053fa03d15b29ba2de2aa7e', 'Content-Length': '0'}
DEBUG:botocore.parsers:Response body:
b''
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <botocore.retryhandler.RetryHandler object at 0x7f7b58b0eed0>
DEBUG:botocore.retryhandler:No retry needed.
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <bound method S3RegionRedirectorv2.redirect_from_error of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
INFO:root:STAC_DEM/Data/bc_092o028_xli1m_utm10_2018_cog.tif
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function validate_bu

Set permissions on STAC_DEM/Data/bc_092o020_xli1m_utm10_2018_cog.tif success, set to public-read
Set permissions on STAC_DEM/Data/bc_092o028_xli1m_utm10_2018_cog.tif success, set to public-read
Set permissions on STAC_DEM/Data/bc_092o029_xli1m_utm10_2019_cog.tif success, set to public-read


DEBUG:botocore.auth:StringToSign:
AWS4-HMAC-SHA256
20240704T154248Z
20240704/us-east-1/s3/aws4_request
633603ff27462479bf8e912666744ba461707912beb94c02c348bb070c6d06f5
DEBUG:botocore.auth:Signature:
453cd6d522bfad8d60b54d78a4da1e15da060bbcf5eae988ed6fa04dc9fee697
DEBUG:botocore.hooks:Event request-created.s3.PutObjectAcl: calling handler <function add_retry_headers at 0x7f7b5b01c900>
DEBUG:botocore.endpoint:Sending http request: <AWSPreparedRequest stream_output=False, method=PUT, url=https://nrs.objectstore.gov.bc.ca:443/cloudgistest/STAC_DEM/Data/bc_092o030_xli1m_utm10_2018_cog.tif?acl, headers={'x-amz-acl': b'public-read', 'User-Agent': b'Boto3/1.34.115 md/Botocore#1.34.115 ua/2.0 os/linux#5.15.153.1-microsoft-standard-WSL2 md/arch#x86_64 lang/python#3.11.7 md/pyimpl#CPython cfg/retry-mode#legacy Botocore/1.34.115', 'Content-MD5': b'1B2M2Y8AsgTpgAmY7PhCfg==', 'X-Amz-Date': b'20240704T154248Z', 'X-Amz-Content-SHA256': b'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855

Set permissions on STAC_DEM/Data/bc_092o030_xli1m_utm10_2018_cog.tif success, set to public-read


In [75]:

# create STAC Catalog 
catalog = pystac.Catalog(id=catalog_name,
                        description='This catalog is a basic demonstration catalog utilizing using nine Digital Elevation Models, in the cloud optimized geotif(COG) format, that overlap with the point cloud test',
                        href=os.path.join(base_url, f"{catalog_name}.json")
                        )

print(json.dumps(catalog.to_dict(), indent=4))

url=create_url(bucket, catalog_name)
print(F"THIS IS THE URL {url}   !!!!!!!!!!!!!!!!!!!!!!!!")

{
    "type": "Catalog",
    "id": "DEM_Test",
    "stac_version": "1.0.0",
    "description": "This catalog is a basic demonstration catalog utilizing using nine Digital Elevation Models, in the cloud optimized geotif(COG) format, that overlap with the point cloud test",
    "links": [
        {
            "rel": "root",
            "href": "https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/DEM_Test.json",
            "type": "application/json"
        },
        {
            "rel": "self",
            "href": "https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/DEM_Test.json",
            "type": "application/json"
        }
    ]
}
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/DEM_Test   !!!!!!!!!!!!!!!!!!!!!!!!


In [76]:
#get base bucket url
base_url=create_url(bucket, stac_loc)

#loop through s3 DEM url and S3 DEM and create item and assets
for dem, s3_dem in zip(dem_url, DEM_Objects):
    info(dem)
    info(s3_dem)
    name=s3_dem.split('/')[-1].split('.')[0]
    info(name)
    bbox, footprint = get_bbox_and_footprint(dem)
    info("bbox: ", bbox, "\n")
    info("footprint: ", footprint)
    datetime_utc = datetime.now(tz=timezone.utc)
    #predict item href before creation? might be a bad idea, roll the dice need to make sure all hrefs get set to public with Matts function after 
    item_href=os.path.join(base_url, name, f"{name}.json")
    info(f"Item Href is : {item_href}")
    item = pystac.Item(id=name,
                    geometry=footprint,
                    bbox=bbox,
                    datetime=datetime_utc,
                    href= item_href,
                    properties={})
    catalog.add_item(item)
 
    # Modify the S3 path to remove the prefix
    s3_dem_key = os.path.relpath(s3_dem, object_key)
    info(f"s3 dem key : {s3_dem_key}")
    #add asset 
    item.add_asset(
    key=name,
    asset=pystac.Asset(
        href=dem,
        media_type=pystac.MediaType.COG
    )
)

INFO:root:https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_2_cog.tif
INFO:root:STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_2_cog.tif
INFO:root:bc_092o018_xli1m_utm10_2018_2_cog
DEBUG:rasterio.env:Entering env context: <rasterio.env.Env object at 0x7f7b58b04290>
DEBUG:rasterio.env:Starting outermost env
DEBUG:rasterio.env:No GDAL environment exists
DEBUG:rasterio.env:New GDAL environment <rasterio._env.GDALEnv object at 0x7f7b59d39570> created
DEBUG:rasterio._env:GDAL_DATA found in environment.
DEBUG:rasterio._env:PROJ_DATA found in environment.
DEBUG:rasterio._env:Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x7f7b59d39570>.
DEBUG:rasterio.env:Entered env context: <rasterio.env.Env object at 0x7f7b58b04290>
DEBUG:rasterio._base:Sharing flag: 0
DEBUG:rasterio._base:Nodata success: 1, Nodata value: -32767.000000
DEBUG:rasterio._base:Dataset <open DatasetReader name='https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Data/bc_092o018_xli

In [77]:
#get collection bounds
unioned_footprint = None
datetime_list=[]
item_list=[]
for item in catalog.get_all_items():
    datetime_list.append(item.datetime)
    item_list.append(item)
    footprint = item.geometry
# Convert the footprint geometry to a Shapely shape
    footprint_shape = shape(footprint)
    
    # Perform union operation
    if unioned_footprint is None:
        # If unioned_footprint is None (first iteration), initialize it with the first footprint
        unioned_footprint = footprint_shape
    else:
        # Otherwise, perform union with the current footprint
        unioned_footprint = unioned_footprint.union(footprint_shape)

collection_bbox = list(unioned_footprint.bounds)
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])

In [78]:
collection_interval = sorted(datetime_list)
temporal_extent = pystac.TemporalExtent(intervals=[collection_interval])
collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)

In [81]:
colection_id='COG-DEM-Test'
collection_href=os.path.join(base_url, f"{colection_id}.json")

collection = pystac.Collection(id=colection_id,
                               description='Digital Elevation Models for the interior of British Columbia',
                               extent=collection_extent,
                               title='BC-DEM',
                               href=collection_href,
                               license='Apache-2.0')


url=create_url(bucket, f"{colection_id}.json")
print(F"THIS IS THE URL {url}   !!!!!!!!!!!!!!!!!!!!!!!!")

THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/COG-DEM-Test.json   !!!!!!!!!!!!!!!!!!!!!!!!


In [82]:
collection.add_items(item_list)
catalog.add_child(collection)

<Link rel=child target=<Collection id=COG-DEM-Test>>

In [83]:
#view parent data
item.get_parent()

<Collection id=COG-DEM-Test>

In [84]:
#visualize STAC Architecture, be careful when using it on large catalogs, as it will walk the entire tree of the STAC.
catalog.describe()

* <Catalog id=DEM_Test>
    * <Collection id=COG-DEM-Test>
      * <Item id=bc_092o018_xli1m_utm10_2018_2_cog>
      * <Item id=bc_092o018_xli1m_utm10_2018_cog>
      * <Item id=bc_092o019_xli1m_utm10_2018_2_cog>
      * <Item id=bc_092o019_xli1m_utm10_2018_cog>
      * <Item id=bc_092o020_xli1m_utm10_2018_2_cog>
      * <Item id=bc_092o020_xli1m_utm10_2018_cog>
      * <Item id=bc_092o028_xli1m_utm10_2018_cog>
      * <Item id=bc_092o029_xli1m_utm10_2019_cog>
      * <Item id=bc_092o030_xli1m_utm10_2018_cog>
  * <Item id=bc_092o018_xli1m_utm10_2018_2_cog>
  * <Item id=bc_092o018_xli1m_utm10_2018_cog>
  * <Item id=bc_092o019_xli1m_utm10_2018_2_cog>
  * <Item id=bc_092o019_xli1m_utm10_2018_cog>
  * <Item id=bc_092o020_xli1m_utm10_2018_2_cog>
  * <Item id=bc_092o020_xli1m_utm10_2018_cog>
  * <Item id=bc_092o028_xli1m_utm10_2018_cog>
  * <Item id=bc_092o029_xli1m_utm10_2019_cog>
  * <Item id=bc_092o030_xli1m_utm10_2018_cog>


In [85]:
# view the last item added with all of it's properties 
info(json.dumps(item.to_dict(), indent=4))

INFO:root:{
    "type": "Feature",
    "stac_version": "1.0.0",
    "id": "bc_092o030_xli1m_utm10_2018_cog",
    "properties": {
        "datetime": "2024-07-04T15:43:23.544498Z"
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    555803.0,
                    5672369.0
                ],
                [
                    555803.0,
                    5681181.0
                ],
                [
                    559397.0,
                    5681181.0
                ],
                [
                    559397.0,
                    5672369.0
                ],
                [
                    555803.0,
                    5672369.0
                ]
            ]
        ]
    },
    "links": [
        {
            "rel": "root",
            "href": "https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/DEM_Test.json",
            "type": "application/json"
        },
        {
  

In [88]:
print("Catalog HREF: ", catalog.get_self_href())
print("Collection HREF:", collection.get_self_href())
print("Item HREF: ", item.get_self_href())

Catalog HREF:  https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/DEM_Test.json
Collection HREF: https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/COG-DEM-Test/collection.json
Item HREF:  https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o030_xli1m_utm10_2018_cog/bc_092o030_xli1m_utm10_2018_cog.json


In [91]:
#save catalog self relative 

ouput_loc=r"STAC_DEM"
catalog.save(catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED, dest_href=ouput_loc)

In [92]:
in_dir=r"STAC_DEM"
#set up transfer? seemed to work
transfer=boto3.s3.transfer.S3Transfer(s3_client)

for root,dirs,files in os.walk(in_dir):
    in_file=os.path.join(root,files[0])
    #check for file existance
    if os.path.isfile(in_file) is True:
        info(f"in path: {in_file}")
        if catalog_name in files[0]:
            out_file=os.path.join(stac_loc,files[0])
        else:
            
            ex_dir=files[0].split('.')[0]
            out_file=os.path.join(stac_loc,ex_dir,files[0])
            # s3_client.put_object(Bucket=bucket, Key=os.path.join(stac_loc,ex_dir))
            # s3_client.delete_object(Bucket=bucket, Key=os.path.join(stac_loc,ex_dir))
    

        info(f"out path: {out_file}")
        transfer.upload_file(in_file, bucket, out_file)
        info(f"{files[0]} put into {out_file} ")
        
    else:
        info("file does not exist!!!!")
        
    



DEBUG:boto3.s3.transfer:Opting out of CRT Transfer Manager. Preferred client: auto, CRT available: False, Instance Optimized: False.
DEBUG:boto3.s3.transfer:Using default client. pid: 4532, thread: 140170240141120
INFO:root:in path: STAC_DEM/DEM_Test.json
INFO:root:out path: STAC_DEM/Catalog/DEM_Test.json
DEBUG:s3transfer.utils:Acquiring 0
DEBUG:s3transfer.tasks:UploadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x7f7b58b3f190>}) about to wait for the following futures []
DEBUG:s3transfer.tasks:UploadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x7f7b58b3f190>}) done waiting for dependent futures
DEBUG:s3transfer.tasks:Executing task UploadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x7f7b58b3f190>}) with kwargs {'client': <botocore.client.S3 object at 0x7f7b58ae7250>, 'config': <boto3.s3.transfer.TransferConfig object at 0x7f7b58c12610>

In [93]:
catalog_list=[]

response = s3_client.list_objects_v2(Bucket=bucket, Prefix="STAC_DEM/Catalog")

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        catalog_list.append(obj['Key'])
        info(obj['Key'])
        info(f"Object Size {obj['Size']}")
        
else:
    error("No objects found in the bucket.")

DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function set_list_objects_encoding_type_url at 0x7f7b5b1f3c40>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function validate_bucket_name at 0x7f7b5b1f27a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function remove_bucket_from_url_paths_from_model at 0x7f7b5b01c9a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <bound method S3RegionRedirectorv2.annotate_request_context of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <bound method ClientCreator._inject_s3_input_parameters of <botocore.client.ClientCreator object at 0x7f7b58b0b290>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function generate_idempotent_uuid at 0x7f7b5b1f25c0>
DEBUG:botocore.

In [94]:
#make all dem objects public with url
for c in catalog_list:
    set_permissions(bucket, c) # default is public-read
    url=create_url(bucket, c)
    print(F"THIS IS THE URL {url}   !!!!!!!!!!!!!!!!!!!!!!!!")

DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function validate_bucket_name at 0x7f7b5b1f27a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function remove_bucket_from_url_paths_from_model at 0x7f7b5b01c9a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <bound method S3RegionRedirectorv2.annotate_request_context of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <bound method ClientCreator._inject_s3_input_parameters of <botocore.client.ClientCreator object at 0x7f7b58b0b290>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function generate_idempotent_uuid at 0x7f7b5b1f25c0>
DEBUG:botocore.hooks:Event before-endpoint-resolution.s3: calling handler <function customize_endpoint_resolver_builtins at 0x7f7b5b01cb80>
DEBUG:botocore.hooks:Event b

Set permissions on STAC_DEM/Catalog/DEM_Test.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/DEM_Test.json   !!!!!!!!!!!!!!!!!!!!!!!!
Set permissions on STAC_DEM/Catalog/bc_092o018_xli1m_utm10_2018_2_cog/bc_092o018_xli1m_utm10_2018_2_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o018_xli1m_utm10_2018_2_cog/bc_092o018_xli1m_utm10_2018_2_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!
Set permissions on STAC_DEM/Catalog/bc_092o018_xli1m_utm10_2018_cog/bc_092o018_xli1m_utm10_2018_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o018_xli1m_utm10_2018_cog/bc_092o018_xli1m_utm10_2018_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!


DEBUG:urllib3.connectionpool:https://nrs.objectstore.gov.bc.ca:443 "PUT /cloudgistest/STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_2_cog/bc_092o019_xli1m_utm10_2018_2_cog.json?acl HTTP/1.1" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 04 Jul 2024 15:58:53 GMT', 'Server': 'ViPR/1.0', 'x-amz-request-id': '8e22ee18:19021c4d5b4:479af:86e8', 'x-amz-id-2': '72065623582e6beba132705a3fcb993106b507b0821d4cea4fcf354d6e64a52b', 'Content-Length': '0'}
DEBUG:botocore.parsers:Response body:
b''
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <botocore.retryhandler.RetryHandler object at 0x7f7b58b0eed0>
DEBUG:botocore.retryhandler:No retry needed.
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <bound method S3RegionRedirectorv2.redirect_from_error of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function validate_bucket_name at 0x7f7b5

Set permissions on STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_2_cog/bc_092o019_xli1m_utm10_2018_2_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_2_cog/bc_092o019_xli1m_utm10_2018_2_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!
Set permissions on STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_cog/bc_092o019_xli1m_utm10_2018_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_cog/bc_092o019_xli1m_utm10_2018_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!


DEBUG:urllib3.connectionpool:https://nrs.objectstore.gov.bc.ca:443 "PUT /cloudgistest/STAC_DEM/Catalog/bc_092o020_xli1m_utm10_2018_2_cog/bc_092o020_xli1m_utm10_2018_2_cog.json?acl HTTP/1.1" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 04 Jul 2024 15:58:53 GMT', 'Server': 'ViPR/1.0', 'x-amz-request-id': '8e22ee18:19021c4d5b4:484a6:41b', 'x-amz-id-2': '1ebd2710792782bdcb2ca56c4bbab67dd5c90df6d5801403afb9e82175454698', 'Content-Length': '0'}
DEBUG:botocore.parsers:Response body:
b''
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <botocore.retryhandler.RetryHandler object at 0x7f7b58b0eed0>
DEBUG:botocore.retryhandler:No retry needed.
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <bound method S3RegionRedirectorv2.redirect_from_error of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function validate_bucket_name at 0x7f7b5b

Set permissions on STAC_DEM/Catalog/bc_092o020_xli1m_utm10_2018_2_cog/bc_092o020_xli1m_utm10_2018_2_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o020_xli1m_utm10_2018_2_cog/bc_092o020_xli1m_utm10_2018_2_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!
Set permissions on STAC_DEM/Catalog/bc_092o020_xli1m_utm10_2018_cog/bc_092o020_xli1m_utm10_2018_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o020_xli1m_utm10_2018_cog/bc_092o020_xli1m_utm10_2018_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!
Set permissions on STAC_DEM/Catalog/bc_092o028_xli1m_utm10_2018_cog/bc_092o028_xli1m_utm10_2018_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o028_xli1m_utm10_2018_cog/bc_092o028_xli1m_utm10_2018_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!


DEBUG:urllib3.connectionpool:https://nrs.objectstore.gov.bc.ca:443 "PUT /cloudgistest/STAC_DEM/Catalog/bc_092o029_xli1m_utm10_2019_cog/bc_092o029_xli1m_utm10_2019_cog.json?acl HTTP/1.1" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 04 Jul 2024 15:58:53 GMT', 'Server': 'ViPR/1.0', 'x-amz-request-id': '8e22ee18:19021c4d5b4:482c2:9f6', 'x-amz-id-2': 'dbcfe638b70e3b18240796bb7d9c1d06c65d46573d71a2753581775e71561983', 'Content-Length': '0'}
DEBUG:botocore.parsers:Response body:
b''
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <botocore.retryhandler.RetryHandler object at 0x7f7b58b0eed0>
DEBUG:botocore.retryhandler:No retry needed.
DEBUG:botocore.hooks:Event needs-retry.s3.PutObjectAcl: calling handler <bound method S3RegionRedirectorv2.redirect_from_error of <botocore.utils.S3RegionRedirectorv2 object at 0x7f7b58bc8310>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.PutObjectAcl: calling handler <function validate_bucket_name at 0x7f7b5b1f27

Set permissions on STAC_DEM/Catalog/bc_092o029_xli1m_utm10_2019_cog/bc_092o029_xli1m_utm10_2019_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o029_xli1m_utm10_2019_cog/bc_092o029_xli1m_utm10_2019_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!
Set permissions on STAC_DEM/Catalog/bc_092o030_xli1m_utm10_2018_cog/bc_092o030_xli1m_utm10_2018_cog.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/bc_092o030_xli1m_utm10_2018_cog/bc_092o030_xli1m_utm10_2018_cog.json   !!!!!!!!!!!!!!!!!!!!!!!!
Set permissions on STAC_DEM/Catalog/collection/collection.json success, set to public-read
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/STAC_DEM/Catalog/collection/collection.json   !!!!!!!!!!!!!!!!!!!!!!!!
